In [13]:

import os  # 운영체제 관련 기능 사용 (환경 변수 로드 등)
from dotenv import load_dotenv  # .env 파일에서 환경 변수를 불러오기 위한 라이브러리
from pinecone import Pinecone

In [14]:
# 아래 두개는 필수
import pdfplumber
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Document, Settings, VectorStoreIndex

In [15]:
PINECONE_ENV = os.getenv("PINECONE_ENV")
PINECONE_API_KEY= os.getenv("PINECONE_API_KEY")
HUGGINGFACE_API_KEY= os.getenv("HUGGINGFACE_API_KEY")


In [16]:
llm = HuggingFaceInferenceAPI(
    # 모델을 불러오자(허깅페이스 번개 표시)
    model_name = 'google/gemma-2-9b-it',
    # 토큰을 집어 넣자
    token = HUGGINGFACE_API_KEY, # 리드 할거니까 리드 토큰
    # 템퍼리처 (꾸밈있이 0  또는 꾸밈없이 1)
    temperature = 0.3,
    # 응답할때 얼마나 길게 할지 ( 1024 = 최대 )
    max_new_token = 1024
)

In [17]:
embed_model = HuggingFaceEmbedding(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [18]:
Settings.llm = llm
Settings.embed_model = embed_model

In [19]:
def extract_text_from_pdf(pdf_path):
  text = ''
  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
      text = text + page.extract_text() +'\n'
  return text

In [20]:
pdf_path = '../data/raw/세법.pdf'

In [41]:
extract_text = extract_text_from_pdf(pdf_path)

In [42]:
# 더큐먼트 만드는 형식
document = [Document(text=extract_text)]

In [22]:
# Pinecone 초기화
pinecone_client = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

# 인덱스 이름 지정 (데이터베이스 이름)
PINECONE_INDEX_NAME = 'kevin-tex'

In [23]:
# 기존의 인덱스 디비가 없으면, 디비 생성
# 처음이면 바로 만들고 기존에 있는게 있으면 바로 사용 할 수 있게
from pinecone import ServerlessSpec

In [24]:


if PINECONE_INDEX_NAME not in pinecone_client.list_indexes().names():
  pinecone_client.create_index(    # 파라미터중 네임 디멘션 스펙 총 3개는 필수도
      name=PINECONE_INDEX_NAME,
      dimension=384,   # 몇 차원짜리의 백터냐? (글자를 숫자로 바꾸는데 몇차원짜리를 할 거야?) - 데이터베이스를 만들떄는 차원수를 정확하게 정해줘야 한다
      metric='cosine',  # cosine을 쓰지 않아도 자동 디폴트 값이 코사인 이긴 하다 # 서버리스로 배포 할거다
      spec = ServerlessSpec(cloud="aws", region=PINECONE_ENV))

In [25]:
# Pinecone 벡터 저장소에 연결
from llama_index.vector_stores.pinecone import PineconeVectorStore

In [26]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_client.Index(PINECONE_INDEX_NAME)
)

In [27]:
# Document(문자)를 인덱싱(숫자로) 한다

In [28]:
from llama_index.core import StorageContext

In [29]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex.from_documents(document, storage_context=storage_context)

NameError: name 'document' is not defined

: 

In [53]:
# Pinecon 에 저장된 인덱스로, 쿼리 수행
# 궁금 한건 쿼리 엔진에게 물어보면 된다

query_engine = index.as_query_engine()

In [12]:
response = query_engine.query('소득세')
# 이렇게 하면 내 말을 숫자로 변경 해서 파인콘 서버로 보낸다.
# 파인콘 서버에서 내가 보낸 숫자와 비슷한것으로 바꾼다
# 글자로 바꿔서 자연스럽게 만든다
# 모델과 임베딩은 쌍으로
# 궁합이 잘 맞는 모델과임베딩이 있다.
# openAI를 이용하면 임베딩도 OpenAi

NameError: name 'query_engine' is not defined

In [63]:
print(response)

탈세 관련 내용은 제공된 텍스트에는 없습니다.



